In [1]:
import re
from typing import List, Dict, Any
from langchain.schema.messages import HumanMessage, AIMessage
from langgraph.state import MessagesState
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

# 예시용 카페 메뉴 데이터. 실제로는 menu_db에 벡터화되어 있어야 함.
menu_data = [
    {"menu_name": "아메리카노", "price": "₩3,000", "description": "진한 에스프레소와 물의 조화. 산뜻하고 깔끔한 맛."},
    {"menu_name": "카페라떼", "price": "₩3,500", "description": "에스프레소에 고소한 우유를 더한 부드러운 커피."},
    {"menu_name": "바닐라라떼", "price": "₩4,000", "description": "달콤한 바닐라 시럽이 들어간 라떼."},
    {"menu_name": "콜드브루", "price": "₩4,200", "description": "장시간 우려낸 진한 콜드브루 커피."}
]

# 벡터 DB 준비 - menu_db
def build_menu_db(menu_data):
    docs = []
    for item in menu_data:
        content = f"{item['menu_name']}\n가격: {item['price']}\n설명: {item['description']}"
        docs.append(Document(page_content=content, metadata={"menu_name": item['menu_name']}))
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    db = FAISS.from_documents(docs, embeddings)
    return db

menu_db = build_menu_db(menu_data)

# 1. 상태 정의
class CafeMessagesState(MessagesState):
    def __init__(self, messages: List[Any], history: List[str] = None):
        super().__init__(messages=messages)
        self.history = history if history else []

# 2. 문의 유형 분류 함수
def classify_intent(user_message: str) -> str:
    if any(x in user_message for x in ["추천", "뭐 먹을까", "인기", "추천해"]):
        return "recommend"
    if any(x in user_message for x in ["얼마", "가격", "비용", "비싸", "저렴"]):
        return "price"
    if any(x in user_message for x in ["메뉴", "종류", "있는지", "파나요", "있나요"]):
        return "menu"
    # 디폴트: 메뉴 문의로 취급
    return "menu"

# 3. 벡터 DB 문서에서 구조화된 정보 추출
def extract_menu_info(doc: Document) -> dict:
    content = doc.page_content
    menu_name = doc.metadata.get('menu_name', 'Unknown')
    price_match = re.search(r'₩[\d,]+', content)
    description_match = re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL)
    return {
        "name": menu_name,
        "price": price_match.group(0) if price_match else "가격 정보 없음",
        "description": description_match.group(1).strip() if description_match else "설명 없음"
    }

# 4. 문의 유형별 답변 생성 함수
def answer_menu_query(state: CafeMessagesState, user_message: str) -> str:
    docs = menu_db.similarity_search(user_message, k=4)
    if not docs:
        return "죄송합니다. 해당 메뉴를 찾을 수 없습니다."
    infos = [extract_menu_info(doc) for doc in docs]
    # 메뉴 이름과 설명을 간단히 나열
    reply = "\n".join([f"- {info['name']}: {info['description']} ({info['price']})" for info in infos])
    return f"저희 카페의 관련 메뉴입니다:\n{reply}"

def answer_price_query(state: CafeMessagesState, user_message: str) -> str:
    docs = menu_db.similarity_search("메뉴 가격", k=5)
    infos = [extract_menu_info(doc) for doc in docs]
    reply = "\n".join([f"- {info['name']}: {info['price']}" for info in infos])
    return f"주요 메뉴의 가격은 다음과 같습니다:\n{reply}"

def answer_recommend_query(state: CafeMessagesState, user_message: str) -> str:
    docs = menu_db.similarity_search(user_message, k=3)
    if not docs:
        docs = menu_db.similarity_search("인기 메뉴", k=3)
    infos = [extract_menu_info(doc) for doc in docs]
    reply = "\n".join([f"- {info['name']}: {info['description']} ({info['price']})" for info in infos])
    return f"추천드리는 메뉴입니다:\n{reply}"

# 5. 전체 에이전트 (LangGraph 스타일)
def create_react_agent():
    def agent_step(state: CafeMessagesState) -> CafeMessagesState:
        user_message = state.messages[-1].content if state.messages else ""
        intent = classify_intent(user_message)
        if intent == "recommend":
            answer = answer_recommend_query(state, user_message)
        elif intent == "price":
            answer = answer_price_query(state, user_message)
        else:
            answer = answer_menu_query(state, user_message)
        state.messages.append(AIMessage(content=answer))
        state.history.append(f"User: {user_message}")
        state.history.append(f"AI: {answer}")
        return state
    return agent_step

# 6. 실제 동작 예시 (복붙 후 테스트)
if __name__ == "__main__":
    # 초기 메시지 상태 생성
    state = CafeMessagesState(messages=[])
    agent = create_react_agent()

    # 사용자가 메시지를 입력했다고 가정
    while True:
        user_input = input("손님: ")
        if user_input.strip().lower() in ["exit", "quit"]:
            break
        state.messages.append(HumanMessage(content=user_input))
        state = agent(state)
        print("AI:", state.messages[-1].content)

# requirements.txt 예시
# langchain
# langgraph
# faiss-cpu
# sentence-transformers

ModuleNotFoundError: No module named 'langgraph.state'